In [3]:
import numpy as np
import itertools

In [4]:
from sklearn.datasets import load_boston
boston = load_boston()
X = boston.data
y = boston.target

In [110]:
def norm(x):
    return np.sqrt(np.sum(np.square(x)))


class MyAwesomeRidgeRegression:
    """Class for linear regression with L2-regularization (aka Ridge).
    """

    def __init__(self, num_of_features, reg_constant):
        self.M = num_of_features
        self.reg_constant = reg_constant
        self.init_weights()

    def init_weights(self, seed=None):
        np.random.seed(seed)
        self.W = np.random.normal(size=self.M)
        self.b = 1

    def predict(self, X):
        return np.dot(X, self.W) + self.b

    def loss(self, X, y):
        return np.sum(np.square(y - self.predict(X)))/2/len(y) +\
    self.reg_constant * np.sum(np.square(self.W))

    def loss_grad(self, X, y):
        diff = y - self.predict(X)
        dW = -np.dot(X.T, diff)/len(y) + 2 * self.reg_constant * self.W
        db = -np.sum(diff)/len(y)
        return dW, db

    def check_grad(self, X, y, eps=1e-6):
        """Numerical gradient checking
        """
        # Calculate real gradients
        my_dW, my_db = self.loss_grad(X, y)

        # Backup parameters
        W0 = self.W.copy()
        b0 = self.b

        # Calculate numerical dW
        dW = []
        for i in range(len(W0)):
            self.W[i] = W0[i] + eps
            l1 = self.loss(X, y)

            self.W[i] = W0[i] - eps
            l2 = self.loss(X, y)

            dW.append( (l1-l2)/2/eps )
            self.W[i] = W0[i]
        dW = np.array(dW)

        # Calculate numerical db
        self.b = b0 + eps
        l1 = self.loss(X, y)

        self.b = b0 - eps
        l2 = self.loss(X, y)

        db = (l1-l2)/2/eps
        self.b = b0

        # Check difference
        print('Difference in dW: %3.3g' % (norm(dW - my_dW)/norm(dW + my_dW)))
        print('Difference in db: %3.3g' % (norm(db - my_db)/norm(db + my_db)))

    def train(self, X, y, learning_rate, Xval, yval, early_stopping=10):
        """Train regression with early stopping by validation data.
        Returns training history.
        """
        err = {
            'train': [],
            'val': [],
        }
        assert early_stopping > 1, 'Early stopping must be more than 1!'

        t = 0
        while t < 100000:
            t += 1

            err['train'].append(self.loss(X, y))
            err['val'].append(self.loss(Xval, yval))

            # Early stopping magic
            if t > early_stopping and err['val'][-1] > err['val'][-(early_stopping+1)]:
                print('Stopped at epoch %d.' % t)
                break

            # Gradient descent
            dW, db = self.loss_grad(X, y)
            self.W -= dW * learning_rate
            self.b -= db * learning_rate

        err['train'].append(self.loss(X, y))
        err['val'].append(self.loss(Xval, yval))
        return err


In [5]:
for i in range(X.shape[1]):
    X[:,i]=(X[:,i]-np.mean(X[:,i]))/np.std(X[:,i])

In [6]:
from sklearn.cross_validation import train_test_split 
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.15, random_state=42) 
Xtr.shape, Xval.shape

((430, 13), (76, 13))

In [112]:
self=MyAwesomeRidgeRegression(13,0)
k=self.train(Xtr,ytr,0.1,Xval,yval)
min=k["val"][-1]
reg_min=0

Stopped at epoch 719.


In [113]:
for reg in np.linspace(0.0001,1,10000):
    self=MyAwesomeRidgeRegression(13,reg)
    k=self.train(Xtr,ytr,0.1,Xval,yval)
    if k["val"][-1] < min:
        min=k["val"][-1]
        reg_min=reg
print(min,reg_min)

Stopped at epoch 42.
Stopped at epoch 45.
Stopped at epoch 43.
Stopped at epoch 45.
Stopped at epoch 43.
Stopped at epoch 47.
Stopped at epoch 46.
Stopped at epoch 71.
Stopped at epoch 43.
Stopped at epoch 42.
Stopped at epoch 45.
Stopped at epoch 44.
Stopped at epoch 45.
Stopped at epoch 43.
Stopped at epoch 48.
Stopped at epoch 52.
Stopped at epoch 46.
Stopped at epoch 45.
Stopped at epoch 47.
Stopped at epoch 45.
Stopped at epoch 45.
Stopped at epoch 46.
Stopped at epoch 60.
Stopped at epoch 42.
Stopped at epoch 44.
Stopped at epoch 46.
Stopped at epoch 44.
Stopped at epoch 45.
Stopped at epoch 44.
Stopped at epoch 47.
Stopped at epoch 46.
Stopped at epoch 43.
Stopped at epoch 45.
Stopped at epoch 41.
Stopped at epoch 43.
Stopped at epoch 58.
Stopped at epoch 47.
Stopped at epoch 43.
Stopped at epoch 44.
Stopped at epoch 47.
Stopped at epoch 51.
Stopped at epoch 52.
Stopped at epoch 44.
Stopped at epoch 43.
Stopped at epoch 68.
Stopped at epoch 45.
Stopped at epoch 45.
Stopped at ep

In [115]:
for rate in np.linspace(0.001,5,1000):
    self=MyAwesomeRidgeRegression(13,0.001)
    k=self.train(Xtr,ytr,rate,Xval,yval)

Stopped at epoch 36894.
Stopped at epoch 1884.
Stopped at epoch 1121.
Stopped at epoch 716.
Stopped at epoch 673.
Stopped at epoch 449.
Stopped at epoch 327.
Stopped at epoch 257.
Stopped at epoch 4562.
Stopped at epoch 3874.
Stopped at epoch 194.
Stopped at epoch 182.
Stopped at epoch 2851.
Stopped at epoch 162.
Stopped at epoch 148.
Stopped at epoch 143.
Stopped at epoch 126.
Stopped at epoch 127.
Stopped at epoch 118.
Stopped at epoch 103.
Stopped at epoch 102.
Stopped at epoch 99.
Stopped at epoch 88.
Stopped at epoch 86.
Stopped at epoch 85.
Stopped at epoch 101.
Stopped at epoch 79.
Stopped at epoch 78.
Stopped at epoch 72.
Stopped at epoch 66.
Stopped at epoch 69.
Stopped at epoch 72.
Stopped at epoch 62.
Stopped at epoch 61.
Stopped at epoch 61.
Stopped at epoch 61.
Stopped at epoch 64.
Stopped at epoch 54.
Stopped at epoch 55.
Stopped at epoch 125.
Stopped at epoch 97.
Stopped at epoch 52.
Stopped at epoch 49.
Stopped at epoch 51.
Stopped at epoch 49.
Stopped at epoch 46.
Stop

In [7]:
def norm(x):
    return np.sqrt(np.sum(np.square(x)))


class MyAwesomeLassoRegression:
    """Class for linear regression with L1-regularization (aka Lasso).
    """

    def __init__(self, num_of_features, reg_constant):
        self.M = num_of_features
        self.reg_constant = reg_constant
        self.init_weights()

    def init_weights(self, seed=None):
        np.random.seed(seed)
        self.W = np.random.normal(size=self.M)
        self.b = 1

    def predict(self, X):
        return np.dot(X, self.W) + self.b

    def loss(self, X, y):
        return np.sum(np.square(y - self.predict(X)))/2/len(y) +\
    self.reg_constant * np.sum(np.abs(self.W))

    def loss_grad(self, X, y):
        diff = y - self.predict(X)
        dW = -np.dot(X.T, diff)/len(y) + self.reg_constant * np.sign(self.W)
        db = -np.sum(diff)/len(y)
        return dW, db

    def check_grad(self, X, y, eps=1e-6):
        """Numerical gradient checking
        """
        # Calculate real gradients
        my_dW, my_db = self.loss_grad(X, y)

        # Backup parameters
        W0 = self.W.copy()
        b0 = self.b

        # Calculate numerical dW
        dW = []
        for i in range(len(W0)):
            self.W[i] = W0[i] + eps
            l1 = self.loss(X, y)

            self.W[i] = W0[i] - eps
            l2 = self.loss(X, y)

            dW.append( (l1-l2)/2/eps )
            self.W[i] = W0[i]
        dW = np.array(dW)

        # Calculate numerical db
        self.b = b0 + eps
        l1 = self.loss(X, y)

        self.b = b0 - eps
        l2 = self.loss(X, y)

        db = (l1-l2)/2/eps
        self.b = b0

        # Check difference
        print('Difference in dW: %3.3g' % (norm(dW - my_dW)/norm(dW + my_dW)))
        print('Difference in db: %3.3g' % (norm(db - my_db)/norm(db + my_db)))

    def train(self, X, y, learning_rate, Xval, yval, early_stopping=10):
        """Train regression with early stopping by validation data.
        Returns training history.
        """
        err = {
            'train': [],
            'val': [],
        }
        assert early_stopping > 1, 'Early stopping must be more than 1!'

        t = 0
        while t < 100000:
            t += 1

            err['train'].append(self.loss(X, y))
            err['val'].append(self.loss(Xval, yval))

            # Early stopping magic
            if t > early_stopping and err['val'][-1] > err['val'][-(early_stopping+1)]:
                print('Stopped at epoch %d.' % t)
                break

            # Gradient descent
            dW, db = self.loss_grad(X, y)
            self.W -= dW * learning_rate
            self.b -= db * learning_rate
            for i in range(len(self.W)):
                if (np.abs(self.W[i])<1e-3): 
                    self.W[i]=0
            if (np.abs(self.b)<1e-3):
                    self.b=0
        err['train'].append(self.loss(X, y))
        err['val'].append(self.loss(Xval, yval))
        return err


In [16]:
def polynom (X0, y, Xval0, yval,learning_rate,reg_constant):
    X=X0
    Xval=Xval0
    new_tr=np.ones((X0.shape[0], 1))
    new_val=np.ones((Xval0.shape[0], 1))
    A=[] #shows selectes combinations of variables
    err=[] #shows erros for every degree of polynomial
    self=MyAwesomeLassoRegression(X.shape[1],reg_constant)
    k2=self.train(X,y,learning_rate,Xval,yval)["val"][-1]
    err.append(k2)

    for k in range(2,20):
        print(k)
        comb=list(itertools.combinations_with_replacement(range(X0.shape[1]),k))
        for i in range(len(comb)):
            for j in comb[i]:
                new_tr *= X[:,j] [...,None]
                new_val *= Xval[:,j] [...,None]
                
            Ztr = np.append(X, new_tr, 1)
            Zval = np.append(Xval, new_val, 1)
            
            new_tr=np.ones((X0.shape[0], 1))
            new_val=np.ones((Xval0.shape[0], 1))
            
            self1=MyAwesomeLassoRegression(Ztr.shape[1],reg_constant)
            self2=MyAwesomeLassoRegression(X.shape[1],reg_constant)
           
            k1=self1.train(Ztr,y,learning_rate,Zval,yval)

            if k1["val"][-1]<k2:
                X=Ztr
                Xval=Zval
                A.append(comb[i])
                k2=k1["val"][-1]
                print(k2,k1["val"][-1])
            print (X.shape[1],Ztr.shape[1])
        err.append(k2)
        if err[-2]-err[-1]<0.1:
            print(err[-1])
            break
    
    return (err,A)

In [17]:
polynom(Xtr,ytr,Xval,yval,0.1,0.01)

Stopped at epoch 47.
2
Stopped at epoch 11.
13 14
Stopped at epoch 42.
6.56003526925 6.56003526925
14 14
Stopped at epoch 44.
14 15
Stopped at epoch 46.
14 15
Stopped at epoch 47.
14 15
Stopped at epoch 77.
5.23416169022 5.23416169022
15 15
Stopped at epoch 106.
15 16
Stopped at epoch 504.
15 16
Stopped at epoch 115.
15 16
Stopped at epoch 75.
15 16
Stopped at epoch 147.
15 16
Stopped at epoch 57.
5.21502740447 5.21502740447
16 16
Stopped at epoch 126.
16 17
Stopped at epoch 1273.
5.16531237115 5.16531237115
17 17
Stopped at epoch 439.
4.98543287038 4.98543287038
18 18
Stopped at epoch 111.
4.93036406452 4.93036406452
19 19
Stopped at epoch 102.
19 20
Stopped at epoch 119.
4.45868152503 4.45868152503
20 20
Stopped at epoch 98.
20 21
Stopped at epoch 960.
4.44375741964 4.44375741964
21 21
Stopped at epoch 61.
21 22
Stopped at epoch 86.
4.42671073586 4.42671073586
22 22
Stopped at epoch 362.
22 23
Stopped at epoch 65.
22 23
Stopped at epoch 114.
22 23
Stopped at epoch 580.
22 23
Stopped 

([6.7346793462832322,
  2.5269085990258264,
  2.3513945229962552,
  2.3475161792390016],
 [(0, 1),
  (0, 5),
  (0, 11),
  (1, 1),
  (1, 2),
  (1, 3),
  (1, 5),
  (1, 7),
  (1, 9),
  (2, 8),
  (2, 10),
  (2, 11),
  (2, 12),
  (3, 4),
  (3, 8),
  (3, 9),
  (3, 11),
  (4, 4),
  (4, 5),
  (4, 11),
  (4, 12),
  (5, 5),
  (5, 6),
  (5, 7),
  (7, 12),
  (8, 8),
  (9, 10),
  (9, 11),
  (10, 10),
  (0, 0, 1),
  (0, 0, 3),
  (0, 1, 1),
  (0, 3, 3),
  (0, 3, 8),
  (0, 3, 9),
  (1, 2, 3),
  (2, 5, 6),
  (3, 4, 12),
  (1, 2, 3, 6),
  (1, 3, 4, 11),
  (2, 2, 2, 3)])